In [ ]:
import pandas as pd

In [ ]:
# See database_helpers.py
run database_helpers.py

In [ ]:
# A list of dimensions that can be processed directly from raw fact data
dims_to_process=["year", "functional_group", "commercial_group", "fishing_sector", "end_use_type"]

In [ ]:
# For each of these dimensions, we will pull all the unique values from the raw fact data
# and insert them into their dimension tables, providing unique ids for primary key values
for dim in dims_to_process:
    print("Loading {} . . .".format(dim))
    selectQuery="""
    SELECT DISTINCT "{}" from "Raw_Fact" ORDER BY "{}"
    """.format(dim, dim)
    
    dim_values = pd.DataFrame(select(selectQuery), columns=[dim])
    dim_values.reset_index(inplace=True)
    dim_values.rename(columns={"index":"id"})
    
    column_name=dim.title().replace("_", "") + "Name"
    if dim == "year":
        column_name = dim.title()
    dim_table_name="Dim_{}".format(dim.title())
    insertQuery="""
    INSERT INTO "{}" ("Id", "{}") VALUES %s
    """.format(dim_table_name, column_name)
    
    bulk_insert(insertQuery, dim_values)